# Module 02: Prompt Engineering for Retail/eCommerce

Welcome to Module 02! In this hands-on notebook, you'll learn practical prompt engineering techniques applied to real-world retail/eCommerce scenarios using Walmart data.

## 🎯 Learning Objectives
By the end of this module, you will:
- Master the principles of effective prompt engineering
- Apply zero-shot and few-shot learning techniques
- Create reusable prompts with LangChain templates
- Understand when to use temperature and top_p parameters
- Build practical AI solutions for retail use cases

## 📚 What You'll Build
- Customer review analysis system
- Support ticket classifier
- Product description generator
- Product comparison tool
- Automated review response system (bonus)

Let's get started! 🚀

## Setup: Import Libraries and Initialize Azure OpenAI

**📋 Instructions:**
1. Run this cell to import all required libraries
2. The code will try to load your API key from a `.env` file
3. If no `.env` file exists, you'll be prompted to enter your API key
4. Ensure `walmart_data.json` exists in `../../data/` directory

**💡 What's happening here:**
- We're setting up LangChain to work with Azure OpenAI
- Creating helper functions to make API calls easier
- Loading sample retail data for our exercises

In [ ]:
import os
import json
import getpass
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from dotenv import load_dotenv

load_dotenv()

# Prompt for API key only if not present in environment variables after loading .env
if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter your Azure OpenAI API key: ")

# Initialize Azure OpenAI model
model = AzureChatOpenAI(
    azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    model="gpt-4o",
    api_version="2024-12-01-preview",
    temperature=0.7,
    top_p=1.0,
)

# Load data
with open('../../data/walmart_data.json', 'r') as f:
    walmart_data = json.load(f)

def get_completion(prompt, temperature=0.7):
    """Get completion from Azure OpenAI using LangChain"""
    temp_model = AzureChatOpenAI(
        azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        model="gpt-4o",
        api_version="2024-12-01-preview",
        temperature=temperature,
        top_p=1.0,
    )
    response = temp_model.invoke(prompt)
    return response.content

def get_completion_with_top_p(prompt, top_p=1.0, temperature=1.0):
    """Get completion with custom top_p from Azure OpenAI"""
    temp_model = AzureChatOpenAI(
        azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        model="gpt-4o",
        api_version="2024-12-01-preview",
        temperature=temperature,
        top_p=top_p,
    )
    response = temp_model.invoke(prompt)
    return response.content

print("✅ Setup complete! Data loaded successfully.")
print(f"📦 Products available: {len(walmart_data['products'])}")
print(f"⭐ Customer reviews: {len(walmart_data['customer_reviews'])}")
print(f"🎫 Support tickets: {len(walmart_data['customer_support_tickets'])}")

---

## Exercise 1: Improved Review Analysis Prompt

**🎯 Objective:** Learn how to write effective prompts with clear structure and requirements.

**📝 Your Task:**
Create a well-structured prompt that analyzes customer reviews for actionable insights.

**Your prompt should include:**
1. **Clear Role Definition** - Who is the AI? (e.g., "You are a customer insights analyst...")
2. **Specific Task** - What exactly should it do?
3. **Structured Output Format** - How should the results be presented?
4. **Clear Requirements** - What insights do you want?

**💡 Tips:**
- Think about what would be most useful for a product team
- Use numbered lists or sections for clarity
- Ask for actionable recommendations, not just summaries
- Consider: sentiment, praise, complaints, recommendations, marketing opportunities

**✍️ TODO:** Complete the `improved_prompt` variable below

In [ ]:
# Get reviews for the Samsung TV (Product P001)
tv_reviews = [r for r in walmart_data['customer_reviews'] if r['product_id'] == 'P001']

# ✍️ TODO: Create your improved prompt below
# Replace the placeholder with your well-structured prompt
improved_prompt = f"""
YOUR PROMPT HERE

Hints to include:
- Define the AI's role (customer insights analyst?)
- Specify the task (analyze reviews for what product?)
- List the reviews: {chr(10).join([f"{i+1}. Rating: {r['rating']}/5 - {r['review_text']}" for i, r in enumerate(tv_reviews)])}
- Request structured output (sentiment, praise, complaints, recommendations, etc.)
"""

# Test your prompt
print("YOUR PROMPT:")
print(improved_prompt)
print("\n" + "="*80 + "\n")
print("AI RESPONSE:")
print(get_completion(improved_prompt, temperature=0.3))

---

## Exercise 2: Zero-Shot Support Ticket Classification

**🎯 Objective:** Learn zero-shot learning - teaching the AI to perform a task without providing examples.

**📝 Your Task:**
Create a prompt that classifies support tickets by priority level using only clear definitions.

**Your prompt should include:**
1. **Role Definition** - Customer support ticket triage specialist
2. **Clear Criteria** - Define what makes a ticket CRITICAL, HIGH, MEDIUM, or LOW priority
3. **Ticket Information** - Include the ticket details
4. **Structured Output** - Priority level, reasoning, and suggested response time

**💡 Key Concepts:**
- **Zero-shot** means the AI has no examples - only definitions
- Use `temperature=0` for consistent classification
- Clear criteria make better classifications

**Priority Level Examples:**
- CRITICAL: Safety issues, data breaches, legal concerns
- HIGH: Product defects, significant impact, time-sensitive
- MEDIUM: Shipping delays, minor issues, general inquiries
- LOW: Questions, feature requests, non-urgent

**✍️ TODO:** Complete the `zero_shot_ticket_prompt` below

In [ ]:
ticket = walmart_data['customer_support_tickets'][0]

# ✍️ TODO: Create your zero-shot classification prompt below
zero_shot_ticket_prompt = f"""
YOUR PROMPT HERE

Hints to include:
- Role: Customer support ticket triage specialist
- Define all priority levels (CRITICAL, HIGH, MEDIUM, LOW)
- Ticket details:
  - Type: {ticket['issue_type']}
  - Description: {ticket['description']}
  - Status: {ticket['status']}
- Request: Priority level, reasoning, and suggested response time
"""

print("Zero-Shot Ticket Classification:")
print(get_completion(zero_shot_ticket_prompt, temperature=0))

---

## Exercise 3: Few-Shot Product Description Writer

**🎯 Objective:** Learn few-shot learning - teaching the AI by providing examples of the desired output.

**📝 Your Task:**
Create a prompt with 2-3 example product descriptions that demonstrate the style you want, then have the AI generate a new description following the same pattern.

**Your prompt should include:**
1. **Role Definition** - Product description writer for Walmart's website
2. **2-3 Quality Examples** - Show the exact style, tone, and format you want
3. **Format Placeholders** - Use `{}` for product name, features, and price
4. **Consistent Style** - All examples should follow the same pattern

**💡 Tips:**
- Make examples engaging and customer-focused
- Use emojis, excitement, and benefits (not just features)
- Keep format consistent across all examples
- Use `temperature=0.7` for creative but structured output

**Example Style Ideas:**
- Start with an exciting hook
- Focus on benefits and lifestyle
- Include emojis for visual appeal
- End with a call-to-action or value statement

**✍️ TODO:** Complete the `few_shot_description_prompt` with your examples

In [ ]:
# ✍️ TODO: Create your few-shot prompt with 2-3 examples
few_shot_description_prompt = """
YOUR PROMPT HERE

Structure:
1. Role: You are writing engaging product descriptions for Walmart's website
2. Examples (provide 2-3):
   
   Example 1:
   Product: [Product Name]
   Description: [Your engaging description with emojis, benefits, excitement!]
   
   Example 2:
   Product: [Product Name]
   Description: [Another description in the same style]
   
   [Add Example 3 if you want]

3. Now write for this product:
   Product: {}
   Key Features: {}
   Price: ${}
   Description:
"""

# Test with LEGO set (Product #6 in the dataset)
test_product = walmart_data['products'][5]
prompt = few_shot_description_prompt.format(
    test_product['name'],
    test_product['description'],
    test_product['price']
)

print("Few-Shot Product Description:")
print(get_completion(prompt, temperature=0.7))

---

## Exercise 4: Product Comparison Prompt Template

**🎯 Objective:** Learn to create reusable prompt templates using LangChain's `PromptTemplate`.

**📝 Your Task:**
Build a reusable template that compares any two products with customizable criteria and target audience.

**Your template should include:**
1. **Input Variables** - List all variables needed (product names, prices, descriptions, ratings, criteria, audience)
2. **Template String** - The prompt structure with `{variable}` placeholders
3. **Structured Output** - Head-to-head comparison, recommendations, value analysis

**💡 LangChain Template Basics:**
```python
template = PromptTemplate(
    input_variables=["var1", "var2", ...],
    template="Your prompt with {var1} and {var2}"
)
```

**What makes a good product comparison?**
- Side-by-side feature comparison
- "Best for" recommendations
- Value analysis (worth the price?)
- Clear final recommendation

**✍️ TODO:** Complete the `comparison_template` using LangChain's PromptTemplate

In [ ]:
# ✍️ TODO: Create your product comparison template
comparison_template = PromptTemplate(
    input_variables=[
        # TODO: List all variables you need
        # Hint: You need details for BOTH products, plus criteria and target audience
        # Example: "product1_name", "product1_price", ...
    ],
    template="""
    YOUR TEMPLATE HERE
    
    Hints:
    - Role: Product comparison expert helping Walmart customers
    - Product A details: {product1_name}, ${product1_price}, {product1_desc}, {product1_rating}/5
    - Product B details: {product2_name}, ${product2_price}, {product2_desc}, {product2_rating}/5
    - Comparison criteria: {criteria}
    - Target audience: {target_audience}
    - Output sections: Head-to-Head, Best For, Value Analysis, Final Recommendation
    """
)

# Test with Samsung TV vs AirPods (interesting cross-category comparison!)
product1 = walmart_data['products'][0]  # Samsung TV
product2 = walmart_data['products'][7]  # AirPods

# ✍️ TODO: Format the template with actual product data
prompt = comparison_template.format(
    # Add all your variables here matching what you defined above
    # Example:
    # product1_name=product1['name'],
    # product1_price=product1['price'],
    # ...
)

print("Product Comparison Report:")
print(get_completion(prompt, temperature=0.5))

---

## Exercise 5: Choosing the Right Parameters

**🎯 Objective:** Learn when to use different `temperature` and `top_p` values for different tasks.

### 🔍 Understanding the Parameters:

**Temperature (0.0 to 2.0):**
- **0.0-0.3**: Deterministic, factual, consistent (data extraction, classification)
- **0.4-0.7**: Balanced creativity and consistency (general tasks)
- **0.8-1.5+**: Highly creative, diverse, varied (brainstorming, creative writing)

**Top_p (0.0 to 1.0):**
- **0.1-0.5**: Very focused, limited vocabulary
- **0.6-0.9**: Balanced diversity
- **0.95-1.0**: Maximum diversity

---

### Scenario 1: Data Extraction

**Task:** Extract the product ID from a support ticket.

**🤔 Think about it:**
- Do we want creativity or accuracy?
- Should the output be the same every time?
- What temperature should we use?

**✍️ TODO:** Choose appropriate parameters and explain your reasoning

In [ ]:
ticket = walmart_data['customer_support_tickets'][0]

# ✍️ TODO: Create a prompt to extract product ID
extraction_prompt = f"""
YOUR PROMPT HERE

Task: Extract the product ID from this support ticket
Ticket Description: {ticket['description']}

Hint: The product ID is mentioned in the ticket. Just extract it cleanly.
"""

# ✍️ TODO: Choose appropriate temperature and top_p values
# Think: Should this be creative or deterministic? Why?
temperature = 0.0  # TODO: Is this correct? Change if needed and explain why below
top_p = 1.0        # TODO: Should you adjust this? Explain why below

# Your reasoning:
reasoning = """
TODO: Explain your parameter choices here
- Why did you choose this temperature?
- Is consistency or creativity more important for data extraction?
- What happens if temperature is too high?
"""

print("Scenario 1: Data Extraction")
print(f"Temperature: {temperature}")
print(f"Top_p: {top_p}")
print(f"\n🧠 Your Reasoning:\n{reasoning}\n")

# Test multiple times to verify consistency
print("Testing consistency (should outputs be identical?):")
for i in range(3):
    response = get_completion_with_top_p(extraction_prompt, top_p=top_p, temperature=temperature)
    print(f"  Attempt {i+1}: {response}")

### Scenario 2: Creative Product Bundles

**Task:** Generate creative marketing ideas for product bundles.

**🤔 Think about it:**
- Do we want the same idea every time or diverse options?
- Should the AI be creative and innovative?
- What temperature should we use?

**✍️ TODO:** Choose appropriate parameters and explain your reasoning

In [ ]:
products = walmart_data['products'][:3]

# ✍️ TODO: Create a prompt to generate creative bundle ideas
bundle_prompt = f"""
YOUR PROMPT HERE

Task: Create 3 creative product bundle ideas combining these products:
1. {products[0]['name']} - ${products[0]['price']}
2. {products[1]['name']} - ${products[1]['price']}
3. {products[2]['name']} - ${products[2]['price']}

For each bundle, include:
- Catchy, memorable name
- Value proposition
- Bundle price with discount
- Target customer
"""

# ✍️ TODO: Choose appropriate temperature and top_p for CREATIVITY
# Think: Should this be creative or deterministic? Why?
temperature = 0.0  # TODO: Is this good for creative tasks? Change it!
top_p = 1.0        # TODO: Should you adjust this?

# Your reasoning:
reasoning = """
TODO: Explain your parameter choices here
- Why did you choose this temperature for creative brainstorming?
- Do you want diverse ideas or consistent outputs?
- What's the difference between temperature 0.0 vs 0.9 for this task?
"""

print("Scenario 2: Creative Bundle Generation")
print(f"Temperature: {temperature}")
print(f"Top_p: {top_p}")
print(f"\n🧠 Your Reasoning:\n{reasoning}\n")

# Generate multiple variations to see diversity
print("Generated Bundle Ideas (observe the variation):")
for i in range(3):
    print(f"\n{'='*80}")
    print(f"Variation {i+1}:")
    print('='*80)
    response = get_completion_with_top_p(bundle_prompt, top_p=top_p, temperature=temperature)
    print(response)

---

## 🌟 Bonus Challenge: Automated Review Response System

**🎯 Objective:** Combine everything you've learned into a real-world application!

**📝 Your Challenge:**
Build a class that automatically generates personalized responses to customer reviews based on their rating and sentiment.

**Requirements:**
1. Create a `ReviewResponseSystem` class
2. Use **few-shot learning** with examples of good responses for different ratings
3. Choose appropriate **temperature** (balance consistency with warmth)
4. Handle different **rating levels** (1-5 stars) appropriately

**💡 Design Considerations:**
- **5-star reviews**: Enthusiastic thanks, encourage sharing
- **3-star reviews**: Acknowledge feedback, offer improvement
- **1-star reviews**: Apologize sincerely, offer immediate resolution

**What you'll practice:**
- Object-oriented design
- Few-shot prompt engineering
- Parameter selection
- Real-world application thinking

**This is optional but highly recommended for deepening your understanding!**

**✍️ TODO:** Complete the `ReviewResponseSystem` class below

In [ ]:
# ✍️ BONUS: Create an automated review response system

class ReviewResponseSystem:
    """Automated system for responding to customer reviews"""
    
    def __init__(self):
        # TODO: Initialize Azure OpenAI client using LangChain
        # Hint: Use AzureChatOpenAI with appropriate settings
        self.client = None  # Replace with your AzureChatOpenAI(...) configuration
        
        # TODO: Create few-shot examples for different rating levels
        # Create examples for 5-star, 3-star, and 1-star reviews
        self.examples = [
            {
                "rating": "5",
                "review": "YOUR EXAMPLE 5-STAR REVIEW",
                "response": "YOUR EXAMPLE RESPONSE TO 5-STAR REVIEW"
            },
            # Add more examples for different rating levels
        ]
    
    def generate_response(self, review_text, rating):
        """Generate appropriate response based on review sentiment"""
        
        # TODO: Build few-shot prompt using your examples
        # Format your examples into a string showing the pattern
        examples_text = ""  # Build this from self.examples
        
        # TODO: Create the prompt
        prompt = f"""
        YOUR PROMPT HERE
        
        Hints:
        - Role: Walmart customer service representative
        - Show examples: {examples_text}
        - Current review to respond to: Rating {rating}/5: {review_text}
        - Request: Generate an appropriate response
        """
        
        # TODO: Use the client to generate response
        # response = self.client.invoke(prompt)
        # return response.content
        
        return "TODO: Implement response generation"

# ✍️ TODO: Test your system (uncomment when ready)
# system = ReviewResponseSystem()
# 
# print("🤖 Automated Review Response System\n")
# print("="*80)
# 
# # Test with first 3 reviews from the dataset
# for i, review in enumerate(walmart_data['customer_reviews'][:3], 1):
#     print(f"\n📝 Review {i}:")
#     print(f"   Rating: {review['rating']}/5")
#     print(f"   Review: {review['review_text']}")
#     print(f"\n💬 Generated Response:")
#     response = system.generate_response(review['review_text'], review['rating'])
#     print(f"   {response}")
#     print("\n" + "="*80)

---

## 📚 Summary: Key Takeaways

### What You Learned in Module 02:

#### 1️⃣ **Prompt Engineering Principles**
- ✅ Define clear roles for the AI
- ✅ Structure output with numbered lists and sections
- ✅ Be specific about requirements
- ✅ Request actionable insights, not just summaries

#### 2️⃣ **Zero-Shot Learning**
- ✅ Teach tasks using only clear definitions and criteria
- ✅ No examples needed - just good instructions
- ✅ Use `temperature=0` for consistent, factual tasks
- ✅ Great for classification, extraction, and structured tasks

#### 3️⃣ **Few-Shot Learning**
- ✅ Teach by showing 2-5 examples of desired output
- ✅ Examples demonstrate style, tone, and format
- ✅ More examples = better pattern matching
- ✅ Great for creative tasks with specific styles

#### 4️⃣ **Prompt Templates with LangChain**
- ✅ Create reusable prompts with variables
- ✅ Use `PromptTemplate` for consistency
- ✅ Easy to maintain and modify
- ✅ Perfect for production applications

#### 5️⃣ **Temperature & Top_p Selection**
| Task Type | Temperature | Top_p | Why? |
|-----------|-------------|-------|------|
| Data Extraction | 0.0-0.3 | 1.0 | Consistency, accuracy |
| Classification | 0.0-0.2 | 1.0 | Deterministic results |
| General Writing | 0.5-0.7 | 0.9-1.0 | Balance creativity & structure |
| Creative Brainstorming | 0.8-1.2 | 0.95 | Diverse, innovative ideas |

---

### 🎯 Next Steps:

1. **Compare with Solutions**: Check `prompt-engineering-solution.ipynb` to see different approaches
2. **Experiment**: Try different temperature values and observe the changes
3. **Apply**: Use these techniques in your own projects
4. **Practice**: The more prompts you write, the better you'll get!

---

### 📖 Additional Resources:

- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [LangChain Prompts Documentation](https://python.langchain.com/docs/modules/model_io/prompts/)
- [Azure OpenAI Best Practices](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering)

---

🎉 **Congratulations on completing Module 02!** 

You now have practical skills in prompt engineering that you can apply to real-world AI applications. Keep experimenting and building! 🚀